In [1]:
import copy
import logging
import argparse
import sys, os
import cupy as cp
import numpy as np
from math import pi
import matplotlib.pyplot as plt
from os.path import abspath, join, expanduser
sys.path.append(abspath(join('../..')))

import matplotlib

%matplotlib inline

import matplotlib as mpl
mpl.use('Qt5Agg')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from skimage import measure

from LevelSetPy.Grids import createGrid
from LevelSetPy.Visualization import *
from LevelSetPy.DynamicalSystems import *
from LevelSetPy.InitialConditions import shapeCylinder
from LevelSetPy.Utilities import *
from LevelSetPy.ExplicitIntegration import *
from LevelSetPy.SpatialDerivative import *
from LevelSetPy.Visualization.value_viz import ValueVisualizer

In [2]:
args = Bundle({'pause_time': 1, 'visualize': True, 'elevation': 5, 'azimuth': 5})

In [3]:
# Grid
grid_min = expand(np.array((-.75, -1.25, -pi)), ax = 1)
grid_max = expand(np.array((3.25, 1.25, pi)), ax = 1)
pdDims = 2                      # 3rd dimension is periodic
resolution = 100
N = np.array(([[resolution, np.ceil(resolution*(grid_max[1, 0] - grid_min[1, 0])/ \
                (grid_max[0, 0] - grid_min[0, 0])), resolution-1 ]])).T.astype(int)
grid_max[2, 0] *= (1-2/N[2])
grid = createGrid(grid_min, grid_max, N, pdDims)

# value func params
axis_align, center, radius = 2, np.zeros((3, 1)), 0.5
value_init = shapeCylinder(grid, axis_align, center, radius)

u_bound, w_bound = 1, 1
drel = DubinsVehicleRel(grid, u_bound, w_bound)

In [4]:
small = 100*eps
level = 0

finite_diff_data = Bundle({'innerFunc': termLaxFriedrichs,
                          'innerData':Bundle({'grid': grid, 
                          'hamFunc': drel.hamiltonian,
                          'partialFunc': drel.dissipation,
                          'dissFunc': artificialDissipationGLF,
                          'CoStateCalc': upwindFirstENO2,
                          }),
                          'positive': True})

options = Bundle(dict(factorCFL=0.95, stats='on', singleStep='off'))
integratorOptions = odeCFLset(options)

In [6]:

# Period at which intermediate plots should be produced.
num_steps = 11
time_interval = [0, 2.5]
t_plot = (time_interval[1] - time_interval[0]) / (num_steps - 1)
# Loop through time_interval (subject to a little roundoff).
t_now = time_interval[0]
start_time = cputime()

# Visualization paramters
spacing = tuple(grid.dx.flatten().tolist())
init_mesh = implicit_mesh(value_init, level=0, spacing=spacing, edge_color='b', face_color='b')

value_init = cp.asarray(value_init)
params = Bundle(
        {"grid": grid,
         'disp': False,
         'labelsize': 16,
         'labels': "Initial 0-LevelSet",
         'linewidth': 2,
         'data': value_init,
         'elevation': args.elevation,
         'azimuth': args.azimuth,
         'mesh': init_mesh,
         'init_conditions': False,
         'pause_time': args.pause_time,
         'level': 0, # which level set to visualize
         'winsize': (16,9),
         'fontdict': Bundle({'fontsize':12, 'fontweight':'bold'}),
         "savedict": Bundle({"save": False,
                        "savename": "rcbrt",
                        "savepath": "/opt/LevPy/Dubins"})
         })

if args.visualize:
    rcbrt_viz = RCBRTVisualizer(params=params)

ValueError: Input volume should be a 3D numpy array.

: 

In [10]:
while(time_interval[1] - t_now > small * time_interval[1]):

    time_step = f"{t_now}/{time_interval[-1]}"

    # Reshape data array into column vector for ode solver call.
    y0 = value_init.flatten()

    # How far to step?
    t_span = np.hstack([ t_now, min(time_interval[1], t_now + t_plot) ])

    # Take a timestep.
    t, y, _ = odeCFL2(termRestrictUpdate, t_span, y0, integratorOptions, finite_diff_data)
    t_now = t

    info(f't: {t:.3f}/{time_interval[-1]} TargSet Min: {min(y):.3f}, TargSet Max: {max(y):.3f} TargSet Norm: {np.linalg.norm(y)}')

    # Get back the correctly shaped data array
    data = np.reshape(y, grid.shape)

    mesh=implicit_mesh(data, level=0, spacing=spacing,  edge_color='None', face_color='red')

    if args.visualize:
        rcbrt_viz.update_tube(data, mesh, time_step)

end_time = cputime()
info(f'Total execution time {end_time - start_time} seconds.')


AttributeError: 'DubinsVehicleRel' object has no attribute 'v_e'